In [1]:
!python -m pip install fastapi uvicorn python-multipart opencv-python easyocr torch torchvision ultralytics


  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 21.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------- ----------------------------- 10.7/39.5 MB 56.1 MB/s eta 0:00:01
   --------------------- ------------------ 21.0/39.5 MB 53.1 MB/s eta 0:00:01
   -----------------------

In [ ]:
from fastapi import FastAPI, UploadFile, Form
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import base64
import cv2
import numpy as np
import easyocr
from ultralytics import YOLO
import io

app = FastAPI()

# Allow all origins (for testing)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Load models
model = YOLO("yolov8n.pt")  # Replace with your custom license plate model
reader = easyocr.Reader(["en"])

class ImageInput(BaseModel):
    image: str  # base64 encoded string

@app.post("/detect")
async def detect_plate(data: ImageInput):
    try:
        # Decode base64
        header, encoded = data.image.split(",", 1)
        image_bytes = base64.b64decode(encoded)
        nparr = np.frombuffer(image_bytes, np.uint8)
        frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

        # Run YOLO
        results = model(frame)[0]

        plate_texts = []
        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            cropped = frame[y1:y2, x1:x2]

            # OCR
            ocr_result = reader.readtext(cropped)
            text = " ".join([i[1] for i in ocr_result])
            if text:
                plate_texts.append(text)

            # Draw box and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.6, (0, 255, 0), 2)

        # Encode final image as base64
        _, buffer = cv2.imencode(".jpg", frame)
        base64_img = base64.b64encode(buffer).decode("utf-8")
        base64_img = f"data:image/jpeg;base64,{base64_img}"

        return {
            "processed_image_base64": base64_img,
            "plates": plate_texts
        }

    except Exception as e:
        return {"error": str(e)}
